# KCM Analyse database

## Query voor resultaten over 1 maand

In [ ]:
select * from dataanalysis.vw_staedion_kpi_scores_samengevat where Jaar = 2022 and Maand = 1 order by KCM_onderzoek asc

## Opbouw view vw\_staedion\_kpi\_scores\_samengevat

In [ ]:
--drop view dataanalysis.vw_staedion_kpi_scores_samengevat 
--CREATE OR REPLACE VIEW dataanalysis.vw_staedion_kpi_scores_samengevat AS
/* --------------------------------------------------------------------------------------------
VAN: JvdW Staedion, 4-1-2022
BETREFT: ophalen benodigde informatie om kpi-scores te kunnen berekenen
Daarvoor is nodig de vraag van KCM + vertaling naar unieke score in dashboard
+ datum ingevulde enquete

NB: questiontypedefinitionid = 5: dan ja/nee vraag - wordt in de view omgezet naar 1/0
NB: sommige vragen verplicht andere niet, delen door aantal ingevulde enquetes op betreffende vraag geeft score
NB: 4-1-2022 data gecheckt met KCM zelf = akkoord
-------------------------------------------------------------------------------------------- */
SELECT Jaar
	, Maand
    , KCM_onderzoek
	, Indicator_Staedion
    , KCM_vraagtypedefinitie_intern
    , max(KCM_vraag) AS KCM_vraag
    , count(*) AS n
    , sum(KCM_resultaat) / count(*) AS Score
    , case when count(DISTINCT KCM_vraag)>1 then 'Vraag is gewijzigd' else 'Vraag niet gewijzigd' end as Opmerking_1
-- select *
FROM dataanalysis.vw_staedion_kpi_scores_details
WHERE Indicator_Staedion IS NOT NULL
--and questiontypedefinitionid = 5
GROUP BY Jaar
	, Maand
	, KCM_onderzoek
    , Indicator_Staedion
    , KCM_vraagtypedefinitie_intern
ORDER BY Jaar
	, KCM_onderzoek
    , Indicator_Staedion
    , KCM_vraagtypedefinitie_intern


## Opbouw view vw_staedion_kpi_scores_details

In [ ]:
-- drop VIEW dataanalysis.vw_staedion_kpi_scores_details
--CREATE OR REPLACE VIEW dataanalysis.vw_staedion_kpi_scores_details AS
/* --------------------------------------------------------------------------------------------
VAN: JvdW Staedion, 4-1-2022
BETREFT: ophalen benodigde informatie om kpi-scores te kunnen berekenen
Daarvoor is nodig de vraag van KCM + vertaling naar unieke score in dashboard
+ datum ingevulde enquete

NB: questiontypedefinitionid = 5: dan ja/nee vraag - wordt in de view omgezet naar 1/0
NB: sommige vragen verplicht andere niet, delen door aantal ingevulde enquetes op betreffende vraag geeft score
NB: 4-1-2022 data gecheckt met KCM zelf = akkoord, alleen STN417 leverde afwijking op
> 35 reacties in 2021 op vraag online terug kunnen vinden
> 89 reacties in 2021 op telefonisch niet terug kunnen vinden
NB
-------------------------------------------------------------------------------------------- */
 SELECT "left"((sr.externalreference)::text, 6) AS KCM_onderzoek,
    qr.questiontext AS KCM_vraag,
        CASE qr.questiondefinitionid
            WHEN 3746 THEN 'Score Planmatig Onderhoud'::text
            WHEN 3760 THEN 'Score huurbetaling'::text
            WHEN 3776 THEN 'Score dagelijks onderhoud'::text
            WHEN 3790 THEN 'Score dagelijks onderhoud'::text
            WHEN 7834 THEN 'Score dagelijks onderhoud'::text
            WHEN 7901 THEN 'Score dagelijks onderhoud'::text			
            WHEN 7819 THEN 'Score dagelijks onderhoud'::text					
            WHEN 3817 THEN 'Score BKT'::text
            WHEN 3868 THEN 'Score groenonderhoud'::text
            WHEN 3887 THEN 'Score telefonische dienstverlening Woonservice'::text
            WHEN 3922 THEN 'Score telefonische dienstverlening Intake'::text
            WHEN 3925 THEN 'Score betrekken woning'::text
            WHEN 3950 THEN 'Score verlaten woning'::text
            WHEN 3968 THEN 'Score klachtenmanagement'::text
            WHEN 6871 THEN 'Indicator Voelt zich thuis'::text
            WHEN 6872 THEN 'Score thuisgevoel'::text
            WHEN 6875 THEN 'Score woningkwaliteit'::text
            WHEN 6885 THEN 'Score algemene ruimte'::text
            WHEN 6892 THEN 'Score buurt'::text
            WHEN 6945 THEN 'Indicator Voelt zich thuis in buurt'::text
            WHEN 6958 THEN 'Indicator schoonmaakonderhoud'::text
            WHEN 7803 THEN 'Indicator huurbetaling'::text			
            ELSE NULL::text
        END AS Indicator_Staedion,
    sr.completiondate AS Datum_ingevuld,
    date_part('year'::text, sr.completiondate) AS jaar,
    date_part('month'::text, sr.completiondate) AS maand,
    ("substring"((qd.externalreference)::text, 8, 10))::integer AS KCM_vraagnummer_intern,
    qr.questiondefinitionid AS KCM_vraagdefinitie_intern,
    qd.questiontypedefinitionid AS KCM_vraagtypedefinitie_intern,
    ar.answertext AS KCM_antwoord,
        CASE
            WHEN ((qd.questiontypedefinitionid = 5) AND (ar.answertext <> 'n / a'::text)) THEN (
            CASE
                WHEN (ar.answertext = 'Ja'::text) THEN 1
                ELSE 0
            END)::numeric
            ELSE
            CASE
                WHEN ((qd.questiontypedefinitionid = 4) AND (ar.answertext <> 'n / a'::text)) THEN (ar.answertext)::numeric
                ELSE NULL::numeric
            END
        END AS KCM_resultaat,
    sr.respondentid,
    sr.surveyresultid
	-- select distinct qr.questiondefinitionid, qr.questiontext
   FROM (((dataanalysis.surveyresults sr
     LEFT JOIN dataanalysis.questionresults qr ON ((sr.surveyresultid = qr.surveyresultid)))
     JOIN dataanalysis.questiondefinitions qd ON ((qr.questiondefinitionid = qd.questiondefinitionid)))
     JOIN dataanalysis.answerresults ar ON ((qr.questionresultid = ar.questionresultid)))
  WHERE (("left"((sr.externalreference)::text, 6) = ANY (ARRAY['STN407'::text, 'STN410'::text, 'STN414'::text, 'STN417'::text, 'STN418'::text, 'STN420'::text, 'STN421'::text, 'STN647'::text, 'STN658'::text, 'STN659'::text, 'STN660'::text, 'STN661'::text, 'STN812'::text, 'STN813'::text, 'STN814'::text])) 
		 AND (date_part('year'::text, sr.completiondate) >= (2020)::double precision) 
		 AND (qd.questiontypedefinitionid = ANY (ARRAY[4, 5])))
		 AND "left"((sr.externalreference)::text, 6) = 'STN814'
  ORDER BY sr.completiondate DESC;


## Opbouw view vw\_staedion\_enquete\_details

In [ ]:
-- drop VIEW dataanalysis.vw_staedion_enquete_details
-- CREATE OR REPLACE VIEW dataanalysis.vw_staedion_enquete_details AS
/* --------------------------------------------------------------------------------------------
VAN: JvdW Staedion, 11-1-2022
BETREFT: ophalen benodigde informatie die gekoppeld is aan ingevulde enquetes
-------------------------------------------------------------------------------------------- */
-- ingevulde surveys
select left(sr.externalreference,6) AS KCM_onderzoek,
		sr.completiondate as Datum_ingevuld,
        date_part('year'::text, sr.completiondate) AS jaar,
        date_part('month'::text, sr.completiondate) AS maand,		
		sr.respondentid, 
		sr.surveyresultid, 
		r.contactid, 
		cfd.name, 
		cf.contactfielddefinitionid, 
		cf.fieldvalue
from dataanalysis.surveyresults  as sr
left outer join dataanalysis.respondent as r
on r.respondentid = sr.respondentid
join dataanalysis.contactfields as cf 
on cf.contactid = r.contactid
join dataanalysis.contactfielddefinitions as cfd 
on cfd.contactfielddefinitionid = cf.contactfielddefinitionid
where extract(year from completiondate) >= 2022-- limit 100
-- and left(sr.externalreference,6) = 'STN660' 
--and cfd.contactfielddefinitionid  in (20,22,88,91)
--and cf.contactid = 976657